# Курсовая

## Шаг 0: **Загрузка данных**

In [1]:
import os
import ast
import sys
from datasets import load_dataset
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
# dataset принимает исходный файл с набором данных
# split принимает часть датасета, которую нужно сохранить (train, test)
# folder_name создает папку с заданным именем, где хранятся таблицы из одного набора данных (обычно train и test)
def save_dataset(dataset, split, folder_name, base_dir="data/raw"): # по умолчанию сохраняем в директорию с "сырыми" датасетами
    base_path = os.path.join(base_dir, folder_name)
    os.makedirs(base_path, exist_ok=True)
    csv_path = os.path.join(base_path, f"{split}.csv")
    df = pd.DataFrame(dataset[split])     
    df.to_csv(csv_path, index=False)
    print(f"{folder_name}/{split}.csv успешно сохранен")

### Датасет: **TweetEval: Sentiment**

In [31]:
tweet_ds = load_dataset("tweet_eval", "sentiment", trust_remote_code=True) 

tweet_train = pd.DataFrame(tweet_ds["train"])
tweet_test = pd.DataFrame(tweet_ds["test"])
tweet_df = pd.concat([tweet_train, tweet_test], ignore_index=True)

word_counts = tweet_df["text"].apply(lambda x: len(str(x).split()))

print(f"Статистика по текстам датасета TweetEval: Sentiment:")
print(f"Мининимум: {word_counts.min()} слов; Среднее: {word_counts.mean()} слов; Максимум: {word_counts.max()} слов")

Статистика по текстам датасета TweetEval: Sentiment:
Мининимум: 1 слов; Среднее: 18.308606366258484 слов; Максимум: 35 слов


In [32]:
# необходимо сузить диапазон до заданного в курсовой
before = tweet_df["label"].value_counts(normalize=True) * 100 # пропорции до
tweet_df = tweet_df[tweet_df["text"].str.strip().str.split().str.len().between(5, 30)].reset_index(drop=True) # сужаем диапазон до заданного
after = tweet_df["label"].value_counts(normalize=True) * 100 # пропорции после

comparison = pd.concat([before, after], axis=1, keys=["До, %", "После, %"])
comparison["Отклонение, %"] = (comparison["После, %"] - comparison["До, %"])
comparison = comparison.round(2)

print("Пропорции классов: до и после")
comparison

Пропорции классов: до и после


,"До, %","После, %","Отклонение, %"
label,,,
1,45.96,45.92,-0.04
2,34.93,34.94,0.01
0,19.11,19.13,0.02


In [33]:
# проверим что фильтрация применилась
word_counts = tweet_df["text"].apply(lambda x: len(str(x).split()))

print(f"Статистика по текстам датасета TweetEval: Sentiment:")
print(f"Мининимум: {word_counts.min()} слов; Среднее: {word_counts.mean()} слов; Максимум: {word_counts.max()} слов")

Статистика по текстам датасета TweetEval: Sentiment:
Мининимум: 5 слов; Среднее: 18.35560041065618 слов; Максимум: 30 слов


In [34]:
# дополнительно сбланарсируем классы
min_count = tweet_df["label"].value_counts().min()

tweet_balanced = tweet_df.groupby("label", group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)

tweet_balanced["label"].value_counts(normalize=True) * 100 # пропорции после балансировки

/var/folders/wv/579phdqx14xch2yg0qz2qr940000gn/T/ipykernel_93474/2345237729.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tweet_balanced = tweet_df.groupby("label", group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)


label
0    33.333333
1    33.333333
2    33.333333
Name: proportion, dtype: float64

In [35]:
# сохраним результат в таблицы
tweet_train, tweet_test = train_test_split(tweet_balanced, test_size=0.3, stratify=tweet_balanced["label"], random_state=42) # вручную разбиваем на train и test
dataset = {"train": tweet_train, "test": tweet_test} # создаем общий датасет чтобы передать в функцию

save_dataset(dataset, "train", "tweet_eval_sentiment") 
save_dataset(dataset, "test", "tweet_eval_sentiment")

tweet_eval_sentiment/train.csv успешно сохранен
tweet_eval_sentiment/test.csv успешно сохранен


### Датасет **AG News**

In [36]:
agn_ds = load_dataset("ag_news", trust_remote_code=True) 

agn_train = pd.DataFrame(agn_ds["train"])
agn_test = pd.DataFrame(agn_ds["test"])
agn_df = pd.concat([agn_train, agn_test], ignore_index=True)

word_counts = agn_df["text"].apply(lambda x: len(str(x).split()))

print(f"Статистика по текстам датасета AG News:")
print(f"Мининимум: {word_counts.min()} слов; Среднее: {word_counts.mean()} слов; Максимум: {word_counts.max()} слов")

Статистика по текстам датасета AG News:
Мининимум: 8 слов; Среднее: 37.84 слов; Максимум: 177 слов


In [37]:
# необходимо сузить диапазон до заданного в курсовой
before = agn_df["label"].value_counts(normalize=True) * 100 # пропорции до
agn_df = agn_df[agn_df["text"].str.strip().str.split().str.len().between(30, 120)].reset_index(drop=True) # сужаем диапазон до заданного
after = agn_df["label"].value_counts(normalize=True) * 100 # пропорции после

comparison = pd.concat([before, after], axis=1, keys=["До, %", "После, %"])
comparison["Отклонение, %"] = (comparison["После, %"] - comparison["До, %"])
comparison = comparison.round(2)

print("Пропорции классов: до и после")
comparison

Пропорции классов: до и после


,"До, %","После, %","Отклонение, %"
label,,,
2,25.0,25.95,0.95
3,25.0,23.09,-1.91
1,25.0,25.63,0.63
0,25.0,25.33,0.33


In [38]:
# проверим что фильтрация применилась
word_counts = agn_df["text"].apply(lambda x: len(str(x).split()))

print(f"Статистика по текстам датасета AG News:")
print(f"Мининимум: {word_counts.min()} слов; Среднее: {word_counts.mean()} слов; Максимум: {word_counts.max()} слов")

Статистика по текстам датасета AG News:
Мининимум: 30 слов; Среднее: 40.465166049563926 слов; Максимум: 120 слов


In [39]:
# дополнительно сбланарсируем классы
min_count = agn_df["label"].value_counts().min()

agn_balanced = agn_df.groupby("label", group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)

agn_balanced["label"].value_counts(normalize=True) * 100 # пропорции после балансировки

/var/folders/wv/579phdqx14xch2yg0qz2qr940000gn/T/ipykernel_93474/1152321853.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agn_balanced = agn_df.groupby("label", group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)


label
0    25.0
1    25.0
2    25.0
3    25.0
Name: proportion, dtype: float64

In [41]:
# сохраним результат в таблицы
agn_train, agn_test = train_test_split(agn_balanced, test_size=0.3, stratify=agn_balanced["label"], random_state=42) # вручную разбиваем на train и test
dataset = {"train": agn_train, "test": agn_test} # создаем общий датасет чтобы передать в функцию

save_dataset(dataset, "train", "ag_news") 
save_dataset(dataset, "test", "ag_news")

ag_news/train.csv успешно сохранен
ag_news/test.csv успешно сохранен


### Датасет **20_Newsgroups**

In [42]:
ng_ds = fetch_20newsgroups(subset="all", remove=('headers','footers','quotes')) # очищаем от нерелевантной информации

ng_df = pd.DataFrame({
    "text": ng_ds.data,
    "label": [ng_ds.target_names[i] for i in ng_ds.target]
})

word_counts = ng_df["text"].apply(lambda x: len(str(x).split()))
        
print(f"Статистика по текстам датасета 20newsgroups:")
print(f"Мининимум: {word_counts.min()} слов; Среднее: {word_counts.mean()} слов; Максимум: {word_counts.max()} слов")

Статистика по текстам датасета 20newsgroups:
Мининимум: 0 слов; Среднее: 181.6377480632495 слов; Максимум: 11765 слов


In [43]:
# сократим диапазон до заданного в курсовой
before = ng_df["label"].value_counts(normalize=True) * 100 # пропорции до
ng_df = ng_df[ng_df["text"].str.strip().str.split().str.len().between(120, 300)].reset_index(drop=True) # сужаем диапазон до заданного
after = ng_df["label"].value_counts(normalize=True) * 100 # пропорции после

comparison = pd.concat([before, after], axis=1, keys=["До, %", "После, %"]) # общая таблица
comparison["Отклонение, %"] = (comparison["После, %"] - comparison["До, %"]) # столбик с отклонением
comparison = comparison.round(2) # округляем до 2 знаков после запятой

print("Пропорции классов: до и после")
comparison

Пропорции классов: до и после


,"До, %","После, %","Отклонение, %"
label,,,
rec.sport.hockey,5.30,5.43,0.13
soc.religion.christian,5.29,6.51,1.22
rec.motorcycles,5.28,4.79,-0.50
rec.sport.baseball,5.27,4.60,-0.68
sci.crypt,5.26,6.59,1.34
rec.autos,5.25,4.81,-0.44
sci.med,5.25,5.95,0.70
comp.windows.x,5.24,5.80,0.56
sci.space,5.24,5.03,-0.21


In [44]:
# проверим что фильтрация применилась
word_counts = ng_df["text"].apply(lambda x: len(str(x).split()))

print(f"Статистика по текстам датасета 20_Newsgroups:")
print(f"Мининимум: {word_counts.min()} слов; Среднее: {word_counts.mean()} слов; Максимум: {word_counts.max()} слов")

Статистика по текстам датасета 20_Newsgroups:
Мининимум: 120 слов; Среднее: 183.87306701030928 слов; Максимум: 300 слов


In [45]:
# дополнительно сбланарсируем классы
min_count = ng_df["label"].value_counts().min()

ng_balanced = ng_df.groupby("label", group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)

ng_balanced["label"].value_counts(normalize=True) * 100 # пропорции после балансировки

/var/folders/wv/579phdqx14xch2yg0qz2qr940000gn/T/ipykernel_93474/761208721.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ng_balanced = ng_df.groupby("label", group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)


label
alt.atheism                 5.0
comp.graphics               5.0
talk.politics.misc          5.0
talk.politics.mideast       5.0
talk.politics.guns          5.0
soc.religion.christian      5.0
sci.space                   5.0
sci.med                     5.0
sci.electronics             5.0
sci.crypt                   5.0
rec.sport.hockey            5.0
rec.sport.baseball          5.0
rec.motorcycles             5.0
rec.autos                   5.0
misc.forsale                5.0
comp.windows.x              5.0
comp.sys.mac.hardware       5.0
comp.sys.ibm.pc.hardware    5.0
comp.os.ms-windows.misc     5.0
talk.religion.misc          5.0
Name: proportion, dtype: float64

In [46]:
ng_train, ng_test = train_test_split(ng_balanced, test_size=0.3, stratify=ng_balanced["label"], random_state=42) # вручную разбиваем на train и test
dataset = {"train": ng_train, "test": ng_test} # создаем общий датасет чтобы передать в функцию

save_dataset(dataset, "train", "20newsgroups") 
save_dataset(dataset, "test", "20newsgroups")

20newsgroups/train.csv успешно сохранен
20newsgroups/test.csv успешно сохранен


## Шаг 0.5: получение оптимальных комбинаций препроцессинга для недостающих моделей

Для получения данных для наших датасетов и моделей модифицируем [исходный код](https://github.com/marco-siino/text_preprocessing_impact/tree/main) авторов [статьи](https://www.sciencedirect.com/science/article/pii/S0306437923001783?ref=cra_js_challenge&fr=RR-1 ) и запустим собственный эксперимент.

С целью не перегружать текущий ноутбук перебор функций препроцессинга для каждого датасета был выполнен в отдельных ноутбуках. Посмотреть их можно, перейдя по ссылкам ниже:
1. [TweetEval: Sentiment](preprocessing/TweetEval_Sentiment.ipynb);
2. [AG News](preprocessing/AG_News.ipynb);
3. [20_Newsgroups](preprocessing/20newsgroups.ipynb);

Кроме того, результаты перебора сохранены в таблицах. Посмотреть их также можно, перейдя по ссылкам ниже:
1. [TweetEval: Sentiment](reports/preprocessing_combinations/tweet_eval_sentiment/full.csv);
2. [AG News](reports/preprocessing_combinations/20newsgroups/full.csv);
3. [20_Newsgroups](reports/preprocessing_combinations/reuters21578/full.csv);

## Шаг 1: Применение оптимальных методов предобработки

In [40]:
import os
import re
import pandas as pd
import numpy as np
import json

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import unicodedata
import contractions
from spellchecker import SpellChecker
import demoji
import wordninja

from pathlib import Path
import html
from joblib import Parallel, delayed

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stepan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt" # загружаем словарь
)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [14]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def correct_spell(text):
    words = text.split()
    corrected = []
    for word in words:
        # получаем предложение по исправлению (VERBOSITY вернет наиболее вероятное)
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2) 
        # если найдено исправление - берем, нет - оставляем исходное слово
        corrected_word = suggestions[0].term if suggestions else word 
        corrected.append(corrected_word)
    return " ".join(corrected)

# нормализация текста (применяется ко всем следующим методам по умолчанию)
def DON(text, dataset_name=None):
    text = html.unescape(text) # конвертируем html-теги в нормальные символы
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode() # удаляем все не-ASCII символы
    text = re.sub(r"<\!\[CDATA\[", " ", text) # удаляем открывающий тег <![CDATA[ из XML
    text = re.sub(r"\]{1,}>", " ", text) # удаляем закрывающие символы ]]
    text = re.sub(r"<[^>]+>", " ", text) # удаляем все HTML и XML теги
    text = re.sub(r"http\S+|www\.\S+", "URL", text) # заменяем ссылки на обозначение URL
    text = re.sub(r"@\w+", "USER", text)  # заменяем упоминания
    text = re.sub(r"#(\w+)", lambda m: ' '.join(wordninja.split(m.group(1))), text) # обрабатываем хэштеги
    text = re.sub(r"[^\w\s]", " ", text) # удаляем спецсимволы и пунктуацию
    text = re.sub(r"\n|\t", " ", text) # удаляем перевод строк и табуляцию
    text = re.sub(r"\s{2,}", " ", text) # удаляем множетсвенные пробелы
    text = re.sub(r"(.)\1{2,}", r"\1\1", text) # удаляем растяжения

    if dataset_name != "ccdv_arxiv-classification": # для научных статей следующие операции бессмысленны + тратят очень ресурсов
        text = demoji.replace_with_desc(text) # заменяем эмодзи текстовым описанием
        text = contractions.fix(text) # заменяем сокращения
        text = correct_spell(text) # исправляем орфографию
    return text.strip()

# приведение к нижнему регистру
def LOW(text):
    return text.lower()

# удаление стоп-слов
def RSW(text):
    words = text.split()
    clear = [w for w in words if w.lower() not in stop_words]
    return " ".join(clear)

# стемминг (приведение к основе слова)
def STM(text):
    words = text.split()
    stemmed = [stemmer.stem(w) for w in words]
    return " ".join(stemmed)

In [49]:
prepo_funcs = {
    "DON": DON,
    "LOW": LOW,
    "RSW": RSW,
    "STM": STM
}

optimal_prepo = {
    "tweet_eval_sentiment": {
        "NB": "RSW - LOW",
        "SVM": "RSW",
        "LR": "RSW - STM - LOW",
        "AdaBoost": "RSW - LOW",
        "XGBoost": "RSW - LOW",
        "RF": "DON",
        "DT": "RSW - STM - LOW"
    },
    "ag_news": {
        "NB": "LOW",
        "SVM": "DON",
        "LR": "DON",
        "AdaBoost": "STM",
        "XGBoost": "RSW",
        "RF": "RSW",
        "DT": "DON"
    },
    "reuters21578_ModLewis": {
        "NB": "RSW - LOW",
        "SVM": "RSW - LOW",
        "LR": "LOW - RSW",
        "AdaBoost": "RSW - LOW",
        "XGBoost": "DON",
        "RF": "RSW - LOW",
        "DT": "RSW"
    },
    "ccdv_arxiv-classification": {
        "NB": "DON",
        "SVM": "DON",
        "LR": "DON",
        "AdaBoost": "RSW - LOW",
        "XGBoost": "RSW",
        "RF": "LOW",
        "DT": "LOW"
    }
}

with open("reports/preprocessing_combinations/optimal_prepo.json", "w") as f:
    json.dump(optimal_prepo, f, indent=2)

In [16]:
source_dir = Path("data/raw")
save_dir = Path("data/processed")

In [17]:
def process_model(dataset_name, model_name, prepo_combs):
    out_dir = save_dir / dataset_name / model_name
    train_path = out_dir / "train_processed.csv"
    test_path = out_dir / "test_processed.csv"

    if train_path.exists() and test_path.exists():
        return f"{dataset_name}: {model_name}: предобработка завершена. Примененные методы: {prepo_combs}"

    train_df = pd.read_csv(source_dir / dataset_name / "train.csv")
    test_df = pd.read_csv(source_dir / dataset_name / "test.csv")

    # парсим наши техники предобработки из словаря
    steps = [s.strip() for s in prepo_combs.split('-')]
    funcs = [prepo_funcs[s] for s in steps]

    # функция применяет каждый метод предобработки, который спарсили из словаря, для текста
    def apply_prepo(text, funcs, dataset_name): 
        text = DON(text, dataset_name=dataset_name)
        if DON not in funcs:
            for f in funcs:
                text = f(text)
        return text

    train_proc = train_df.copy()
    test_proc = test_df.copy()

    # применяем препроцессинг к колонке с текстом
    train_proc["text"] = train_proc["text"].astype(str).apply(lambda x: apply_prepo(x, funcs, dataset_name))
    test_proc["text"] = test_proc["text"].astype(str).apply(lambda x: apply_prepo(x, funcs, dataset_name))

    # сохраняем результат предобработки
    os.makedirs(out_dir, exist_ok=True)
    train_proc.to_csv(train_path, index=False)
    test_proc.to_csv(test_path, index=False)

    return f"{dataset_name}: {model_name}: предобработка завершена. Примененные методы: {prepo_combs}"

In [18]:
# создаем список задач для параллельного запуска
# изначально планировалось использовать хотя 2 ядра, но при n_job=2 озу быстро заканчивается из-за чего машина уходит в перезагрузку
tasks = [
    delayed(process_model)(dataset_name, model_name, prepo_comb)
    for dataset_name, model_map in optimal_prepo.items() # проходимся по датасетам
    for model_name, prepo_comb in model_map.items() # проходимся по моделям и методам предобработки
]

results = Parallel(n_jobs=1, backend="loky")(tasks)
for res in results:
    print(res)

tweet_eval_sentiment: NB: предобработка завершена. Примененные методы: RSW - LOW
tweet_eval_sentiment: SVM: предобработка завершена. Примененные методы: RSW
tweet_eval_sentiment: LR: предобработка завершена. Примененные методы: RSW - STM - LOW
tweet_eval_sentiment: AdaBoost: предобработка завершена. Примененные методы: RSW - LOW
tweet_eval_sentiment: XGBoost: предобработка завершена. Примененные методы: RSW - LOW
tweet_eval_sentiment: RF: предобработка завершена. Примененные методы: DON
tweet_eval_sentiment: DT: предобработка завершена. Примененные методы: RSW - STM - LOW
ag_news: NB: предобработка завершена. Примененные методы: LOW
ag_news: SVM: предобработка завершена. Примененные методы: DON
ag_news: LR: предобработка завершена. Примененные методы: DON
ag_news: AdaBoost: предобработка завершена. Примененные методы: STM
ag_news: XGBoost: предобработка завершена. Примененные методы: RSW
ag_news: RF: предобработка завершена. Примененные методы: RSW
ag_news: DT: предобработка завершена.

## Шаг 2: векторизация текста

In [36]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import joblib
from joblib import load
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec, FastText
import gensim.downloader as api
from scipy import sparse

nltk.download('punkt', quiet=True)

True

In [24]:
source_dir = Path("data/processed")
save_dir = Path("data/vectorized")

In [32]:
max_features = 50000
emb_size = 300
min_count = 5
workers = 4

In [30]:
def tfidf_weighted_avg(tokenized_texts, keyed_vectors, tfidf_vectorizer, dim):
    tfidf_matrix = tfidf_vectorizer.transform([" ".join(tokens) for tokens in tokenized_texts])
    vocab = tfidf_vectorizer.vocabulary_
    result = np.zeros((len(tokenized_texts), dim), dtype=np.float32)

    for i, tokens in enumerate(tokenized_texts):
        vecs = []
        weights = []
        for word in tokens:
            if word in keyed_vectors and word in vocab:
                tfidf_weight = tfidf_matrix[i, vocab[word]]
                vecs.append(keyed_vectors[word])
                weights.append(tfidf_weight)
        if vecs:
            vecs = np.array(vecs)
            weights = np.array(weights)
            result[i] = np.average(vecs, axis=0, weights=weights)
    return result

In [39]:
for dataset_dir in source_dir.iterdir():
    if not dataset_dir.is_dir():
        continue
        
    dataset_name = dataset_dir.name
    print(f"Датасет {dataset_name}")

    for model_dir in dataset_dir.iterdir():
        if not model_dir.is_dir():
            continue
            
        model_name = model_dir.name

        df_train = pd.read_csv(model_dir / "train_processed.csv")
        df_test = pd.read_csv(model_dir / "test_processed.csv")

        text_train = df_train["text"].astype(str).tolist()
        text_test = df_test["text"].astype(str).tolist()

        base_target_dir = save_dir / dataset_name / model_name

        print(f"\n\t{model_name}: начало перебора эмбеддингов\n")

        # BOW и TF-IDF
        for name, vec in [
            ("bow", CountVectorizer(max_features=max_features, ngram_range=(1,2))),
            ("tfidf", TfidfVectorizer(max_features=max_features, ngram_range=(1,2)))
        ]:
            print(f"\t\tТип векторизации: {name}: запущен")

            out_dir = base_target_dir / name
            os.makedirs(out_dir, exist_ok=True)

            if ((out_dir / "train_vectorized.npz").exists() and 
                (out_dir / "test_vectorized.npz").exists() and 
                (out_dir / f"{name}.joblib").exists()):
                print(f"\t\tТип векторизации: {name}: завершен\n")
                continue
                            
            X_train = vec.fit_transform(text_train)
            X_test = vec.transform(text_test)

            sparse.save_npz(out_dir / "train_vectorized.npz", X_train)
            sparse.save_npz(out_dir / "test_vectorized.npz", X_test)
            joblib.dump(vec, out_dir / f"{name}.joblib")

            print(f"\t\tТип векторизации: {name}: завершен\n")

        # Токенизируем текст для перед плотной векторизацией
        tokens_train = [word_tokenize(t) for t in text_train]
        tokens_test = [word_tokenize(t) for t in text_test]

        # WORD2VEC
        for name, sg in [("word2vec_cbow", 0), ("word2vec_sg", 1)]:
            print(f"\t\tТип векторизации: {name}: запущен")

            out_dir = base_target_dir / name
            os.makedirs(out_dir, exist_ok=True)

            if ((out_dir / "train_vectorized.npy").exists() and 
                (out_dir / "test_vectorized.npy").exists() and 
                (out_dir / f"{name}.kv").exists()):
                print(f"\t\tТип векторизации: {name}: завершен\n")
                continue
            
            window = 4 if sg == 0 else 10
            negative = 5 if sg == 0 else 10
            
            w2v = Word2Vec(
                sentences=tokens_train,
                vector_size=emb_size,
                window=window,
                min_count=min_count,
                workers=workers,
                sg=sg,
                epochs=5,
                negative=negative,
                sample=1e-3
            )

            tfidf_vec = load(base_target_dir / "tfidf" / "tfidf.joblib")

            X_train = tfidf_weighted_avg(tokens_train, w2v.wv, tfidf_vec, emb_size)
            X_test = tfidf_weighted_avg(tokens_test, w2v.wv, tfidf_vec, emb_size)

            np.save(out_dir / "train_vectorized.npy", X_train)
            np.save(out_dir / "test_vectorized.npy", X_test)
            w2v.wv.save(str(out_dir / f"{name}.kv"))

            print(f"\t\tТип векторизации: {name}: завершен\n")

        # GLOVE
        print(f"\t\tТип векторизации: GloVe: запущен")

        out_dir = base_target_dir / "glove"
        os.makedirs(out_dir, exist_ok=True)

        if ((out_dir / "train_vectorized.npy").exists() and 
            (out_dir / "test_vectorized.npy").exists() and 
            (out_dir / "glove.kv").exists()):
            print(f"\t\tТип векторизации: glove: завершен\n")
            continue
        
        glove = api.load("glove-wiki-gigaword-300")
        tfidf_vec = load(base_target_dir / "tfidf" / "tfidf.joblib")

        X_train = tfidf_weighted_avg(tokens_train, glove, tfidf_vec, emb_size)
        X_test = tfidf_weighted_avg(tokens_test, glove, tfidf_vec, emb_size)

        np.save(out_dir / "train_vectorized.npy", X_train)
        np.save(out_dir / "test_vectorized.npy", X_test)
        glove.save(str(out_dir / f"glove.kv"))
        print(f"\t\tТип векторизации: GloVe: завершен\n")

        # FASTTEXT
        print(f"\t\tТип векторизации: FastText: запущен")

        out_dir = base_target_dir / "fasttext"
        os.makedirs(out_dir, exist_ok=True)

        if ((out_dir / "train_vectorized.npy").exists() and 
            (out_dir / "test_vectorized.npy").exists() and 
            (out_dir / "fasttext.kv").exists()):
            print(f"\t\tТип векторизации: FastText: завершен\n")
            continue
        
        ft = FastText(
            sentences=tokens_train,
            vector_size=emb_size,
            window=10,
            min_count=min_count,
            workers=workers,
            sg=1,
            epochs=5
        )

        tfidf_vec = load(base_target_dir / "tfidf" / "tfidf.joblib")

        X_train = tfidf_weighted_avg(tokens_train, ft.wv, tfidf_vec, emb_size)
        X_test = tfidf_weighted_avg(tokens_test, ft.wv, tfidf_vec, emb_size)

        np.save(out_dir / "train_vectorized.npy", X_train)
        np.save(out_dir / "test_vectorized.npy", X_test)
        ft.wv.save(str(out_dir / "fasttext.kv"))
        print(f"\t\tТип векторизации: FastText: завершен\n")

Датасет tweet_eval_sentiment

	DT: начало перебора эмбеддингов

		Тип векторизации: bow: запущен
		Тип векторизации: bow: завершен

		Тип векторизации: tfidf: запущен
		Тип векторизации: tfidf: завершен

		Тип векторизации: word2vec_cbow: запущен
		Тип векторизации: word2vec_cbow: завершен

		Тип векторизации: word2vec_sg: запущен
		Тип векторизации: word2vec_sg: завершен

		Тип векторизации: GloVe: запущен
[===-----------------------------------------------] 6.2% 23.4/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[========------------------------------------------] 17.0% 63.9/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[===========---------------------------------------] 22.7% 85.5/376.1MB downloaded

...

[==========================================--------] 85.2% 320.6/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==============================================----] 93.7% 352.2/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




		Тип векторизации: GloVe: завершен

		Тип векторизации: FastText: запущен
		Тип векторизации: FastText: завершен


	XGBoost: начало перебора эмбеддингов

		Тип векторизации: bow: запущен
		Тип векторизации: bow: завершен

		Тип векторизации: tfidf: запущен
		Тип векторизации: tfidf: завершен

		Тип векторизации: word2vec_cbow: запущен
		Тип векторизации: word2vec_cbow: завершен

		Тип векторизации: word2vec_sg: запущен
		Тип векторизации: word2vec_sg: завершен

		Тип векторизации: GloVe: запущен
		Тип векторизации: GloVe: завершен

		Тип векторизации: FastText: запущен
		Тип векторизации: FastText: завершен


	RF: начало перебора эмбеддингов

		Тип векторизации: bow: запущен
		Тип векторизации: bow: завершен

		Тип векторизации: tfidf: запущен
		Тип векторизации: tfidf: завершен

		Тип векторизации: word2vec_cbow: запущен
		Тип векторизации: word2vec_cbow: завершен

		Тип векторизации: word2vec_sg: запущен
		Тип векторизации: word2vec_sg: завершен

		Тип векторизации: GloVe: запущен


## Шаг 3: гибридизация эмбеддингов + обучение моделей

In [79]:
import itertools
import json
import os
import gc
import time
import ast
from pathlib import Path
from collections import defaultdict
import joblib
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [53]:
# загружает векторное представление
# возвращает загруженное векторное представление + метку: 1 - разреженный вектор, 0 - плотный вектор
def load_vector(path):
    if path.suffix == ".npz":
        X = sparse.load_npz(path)
        return X, 1
    elif path.suffix == ".npy":
        X = np.load(path)
        return X, 0

In [54]:
# конкатенируют два векторных представления в зависимости от типа
def convec(X1, vectype1, X2, vectype2):
    if vectype1 and vectype2: # если два векторных представления - разреженные
        return sparse.hstack([X1, X2])
    elif vectype1 and not vectype2: # если первое векторное представление - разреженное, а второе - плотное
        return sparse.hstack([X1, sparse.csr_matrix(X2)])
    elif not vectype1 and vectype2: # если первое векторное представление - плотное, а второе - разреженное
        return sparse.hstack([sparse.csr_matrix(X1), X2])
    else: # если два векторных представления - плотные
        return np.hstack([X1, X2])

In [69]:
def build_model(model_name):
    models = {
        "NB": MultinomialNB(),
        "SVM": LinearSVC(random_state=0),
        "LR": LogisticRegression(random_state=0, max_iter=1000),
        "AdaBoost": AdaBoostClassifier(n_estimators=200, random_state=0),
        "XGBoost": XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, subsample=0.9, tree_method="hist", random_state=0),
        "RF": RandomForestClassifier(n_estimators=200, max_depth=20, random_state=0),
        "DT": DecisionTreeClassifier(max_depth=20, random_state=0)
    }
    return models[model_name]

In [58]:
# функция загружает метки классов
# для датасета reuters21578 создается маска для фильтрации пустых значений меток
def load_labels(csv_path, dataset_name):
    df = pd.read_csv(csv_path)
    if dataset_name == "reuters21578_ModLewis": # единственный многоклассовый (multi-label) датасет, у которого метки в колонке с названием topics 
        # преобразуем строку с темами в массив
        df["topics"] = df["topics"].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else []
        )
        # маска позволяет отфильтровать пустые строки, оставив только те, где topics не пуст
        mask = df["topics"].apply(lambda x: isinstance(x, list) and len(x) > 0)
        df = df[mask].copy()

        # берем первую тему как метку
        df["label"] = df["topics"].apply(lambda x: x[0])
        labels = df["label"].astype(str).values
        return labels, mask.values # маска для фильтрации пустых строк
    else:
        labels = df["label"].astype(str).values
        return labels, np.ones(len(labels), dtype=bool) # маска состоящая из единиц, то есть без фильтрации

In [74]:
solo_vecs = ["bow", "tfidf", "word2vec_cbow", "word2vec_sg", "glove", "fasttext"]
hybrid_pairs = list(itertools.combinations(solo_vecs, 2))

In [50]:
dataset_size = {
    "tweet_eval_sentiment": "very_small",
    "ag_news": "small",
    "reuters21578_ModLewis": "medium",
    "ccdv_arxiv-classification": "large"
}
with open("reports/preprocessing_combinations/optimal_prepo.json") as f:
    optimal_prepo = json.load(f)

In [65]:
vec_root = Path("data/vectorized")
results_dir = Path("reports/vectorizing_combinations")
models_dir = Path("models")
processed_root = Path("data/processed")

In [87]:
def train_model(dataset_name, model_name, prepo_comb, vecs,
              y_train_full, y_test_full, mask_train, mask_test,
              vec1, vec2):
    
    model_path = models_dir / dataset_name / model_name
    model_file = model_path / f"{model_name}_{prepo_comb}_{vec1}-{vec2}.joblib"
    if model_file.exists(): # пропускаем обучение для уже существующих моделей
        return None
        
    vec1_X_train, vec1_X_test, vectype1 = vecs[vec1]
    vec1_X_train = vec1_X_train[mask_train] # фильтруем строки с пустыми темами текстов
    vec1_X_test = vec1_X_test[mask_test]

    vec2_X_train, vec2_X_test, vectype2 = vecs[vec2]
    vec2_X_train = vec2_X_train[mask_train] # фильтруем строки с пустыми темами текстов
    vec2_X_test = vec2_X_test[mask_test]

    y_train = y_train_full[mask_train]
    y_test = y_test_full[mask_test]

    # создаем гибрид путем конкатенации векторных представлений
    X_train = convec(vec1_X_train, vectype1, vec2_X_train, vectype2)
    X_test = convec(vec1_X_test, vectype1, vec2_X_test, vectype2)

    if model_name == "NB": # на всякий случай обрабатываем отрицательные значения 
        scaler = MaxAbsScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    # обучаем модель
    model = build_model(model_name)
    start_time = time.perf_counter() # наиболее точный замер времени
    model.fit(X_train, y_train)
    train_time = time.perf_counter() - start_time

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average="macro")
    f1 = f1_score(y_test, y_pred, average="macro")

    # сохраняем обученную модель
    os.makedirs(model_path, exist_ok=True)
    joblib.dump(model, model_file, compress=3)

    gc.collect() # освобождаем оперативную память после обучения модели на гибриде

    # делаем запись в будущую таблицу
    return {
            "dataset": dataset_name,
            "size": size,
            "model": model_name,
            "prepo_funcs": prepo_comb,
            "hybrid": f"{vec1}+{vec2}",
            "accuracy": round(accuracy, 3),
            "recall": round(recall, 3),
            "f1": round(f1, 3),
            "train_time": round(train_time, 5)
    }

In [92]:
for dataset_dir in vec_root.iterdir():
    if not dataset_dir.is_dir(): # пропускаем системные файлы (macos создает скрытые .DS_Store)
        continue
    dataset_name = dataset_dir.name
    size = dataset_size[dataset_name]
    records = []

    print(f"\n{dataset_name}: начало обучения\n")

    for model_dir in dataset_dir.iterdir():
        if not model_dir.is_dir(): # пропускаем системные файлы (macos создает скрытые .DS_Store)
            continue
        model_name = model_dir.name
        prepo_comb = optimal_prepo[dataset_name][model_name].replace(" - ", "-")
        vecs = {} # словарь всех векторных представлений для данной модели

        # загружаем целевые переменные и маску
        y_train_full, mask_train = load_labels(processed_root / dataset_name / model_name / "train_processed.csv", dataset_name)
        y_test_full, mask_test = load_labels(processed_root / dataset_name / model_name / "test_processed.csv", dataset_name)

        # кодируем метки
        le = LabelEncoder()
        y_train_full = le.fit_transform(y_train_full)
        y_test_full  = le.transform(y_test_full)
        
        for vec_name in solo_vecs: # проходимся по всем одиночным представлениям и записываем в словарь с меткой
            vec_dir = model_dir / vec_name
            
            X_train, vectype1 = load_vector(vec_dir / "train_vectorized.npz" if (vec_dir / "train_vectorized.npz").exists() else vec_dir / "train_vectorized.npy")
            X_test, vectype2 = load_vector(vec_dir / "test_vectorized.npz" if (vec_dir / "test_vectorized.npz").exists() else vec_dir / "test_vectorized.npy")
            
            vecs[vec_name] = (X_train, X_test, vectype1)

        valid_pairs = [p for p in hybrid_pairs if p[0] in vecs and p[1] in vecs] # считаем количество возможных гибридных пар

        # параллельно обучаем все пары, так как последовательно получается долго
        results = Parallel(n_jobs=2, backend="loky")(
            delayed(train_model)(
                dataset_name, model_name, prepo_comb, vecs,
                y_train_full, y_test_full, mask_train, mask_test,
                vec1, vec2
            )
            for vec1, vec2 in valid_pairs
        )
        results = [r for r in results if r is not None]
        records.extend(results)

        skipped = len(valid_pairs) - len(results)
        print(f"\t{model_name}: обработано: {len(results)}/{len(valid_pairs)} гибридов, пропущено: {skipped}/{len(valid_pairs)} гибридов")

    # сохраняем таблицу результатов для датасета
    df = pd.DataFrame(records)
    df.sort_values(["model", "f1"], ascending=[True, False], inplace=True)
    df.to_csv(results_dir / f"{dataset_name}_results.csv", index=False)
    print(f"\n{dataset_name}: результаты обучения успешно сохранены\n")


tweet_eval_sentiment: начало обучения

	DT: обработано: 0/15 гибридов, пропущено: 15/15 гибридов
	XGBoost: обработано: 0/15 гибридов, пропущено: 15/15 гибридов
	RF: обработано: 0/15 гибридов, пропущено: 15/15 гибридов
	SVM: обработано: 0/15 гибридов, пропущено: 15/15 гибридов


/opt/homebrew/anaconda3/envs/conda/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 